# Bibliotecas

In [13]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re
from requests.exceptions import HTTPError
import urllib
from selenium import webdriver
from reppy.robots import Robots
from urllib.parse import urlparse

# Crawler

In [14]:
Dominio = ['https://www.capefeargames.com/','https://www.mtgotraders.com/',
           'https://www.wizardscupboard.com/','https://abugames.com/',
           'https://www.cardkingdom.com/','http://www.starcitygames.com/',
           'https://www.tcgplayer.com/','https://scryfall.com/']

paginasVisitadas = []
paginasRelevante = {'Relevate':[],'Nao-Relevate':[]}

Dados = []

In [15]:
# Função para marcar a page como visitada.
def Visitou (url):
    if url not in paginasVisitadas:
        return False
    else:
        return True

In [16]:
# Função para verificar se a requisição sofreu algum problema.
def verificarRequest(url):
    try:
        response = rq.head(url)
        # O response retornou alguma conexao valida.
        response.raise_for_status()

    except HTTPError as http_err:
        #print(f'HTTP error occurred: {http_err}')
        return False
    except Exception as err:
        #print(f'Other error occurred: {err}')
        return False
    return True

In [17]:
def CrawlerDinâmico(url):
    #C:\\Users\\Usuário\\Downloads\\chromedriver76\\chromedriver.exe 
    driver = webdriver.Chrome("C:\\Users\\Usuário\\Downloads\\chromedriver76\\chromedriver.exe")
    driver.get(url)
    res = driver.execute_script("return document.documentElement.outerHTML")
    driver.quit
    return res

In [18]:
#Função para Verificar se podemos baixar a page ou não
def verificarRobotTxt(url):
    robots = Robots.fetch(url + '/robots.txt')
    return robots.allowed(url, '*')

In [19]:
verificarRobotTxt(Dominio[0])

True

In [20]:
# Criar uma Tabela com as informações sobre links.
def Tabela(count,url,site):
    Dados.append(dict(zip(['Número','Link','Site'],[count,url,site])))

## * Baseline

In [21]:
def CardsCrawler(url):
    print('Dominio: ' + url.geturl())
    count = 0
    paginas = [url]
    #paginasVisitadas = []
    robots = Robots.fetch(url.geturl() + '/robots.txt')
    while count < 1000 and len(paginas) > 0:
        count = count + 1
        primeiraPagina = paginas.pop(0)
        print(count,'-',primeiraPagina.geturl(),' Qtd Paginas -- ', len(paginas))
       
        # Verifica se a pagina foi visitada.Se foi, procuro o proximo link da lista que não foi visitado.
        # Verifico se o hostname de cada pagina é igual ao do dominio, se não, rocuro o proximo link da lista (navegar apenas dentro do site)
        # Verifico se hostname existe , se não , procuro o proximo link que tenha.
        # Verifico se é possivel crawlear a pagina de acordo com as regras do robot.txt , se não , procuro o proximo link.
        while Visitou(primeiraPagina) == True or primeiraPagina.hostname != url.hostname or primeiraPagina.hostname == '' or robots.allowed(primeiraPagina.geturl(), '*') == False:
            if len(paginas) == 0:
                print('lascou@!')
                return
            primeiraPagina = paginas.pop(0)
        
        # verificar se a requisição funciana.
        if verificarRequest(primeiraPagina.geturl()) == True:

            # Verficar se o site é dinamico.
            if (Dominio[3] in url.geturl()) or (Dominio[6] in url.geturl()):
                texto = CrawlerDinâmico(primeiraPagina.geturl())
            else:
                #request - usando o content type
                codigo_fonte = rq.get(primeiraPagina.geturl(),headers={'content-type': 'text/html'})
                # texto do codigo_fonte.
                texto = codigo_fonte.text
            soup = BeautifulSoup(texto,'html.parser')

            # Encontrar todos os links da pagina(url). # attrs={'href': re.compile("^http://")}
            for link in soup.findAll('a'):
                href = link.get('href')
                # Correção de href relativos para absolutos.
                href = urllib.parse.urljoin(url.geturl(),href)
                paginas.append(urlparse(href))

            # Marco como visitada a urlparse    
            paginasVisitadas.append(primeiraPagina)

        else:
            print('Requisição falhou!')
        
        Tabela(count,primeiraPagina.geturl(),url.geturl())

In [ ]:
for url in Dominio:
    CardsCrawler(urlparse(url))

Dominio: https://www.capefeargames.com/
1 - https://www.capefeargames.com/  Qtd Paginas --  0
2 - mailto:info@capefeargames.com  Qtd Paginas --  869
3 - https://www.capefeargames.com/faq__policies  Qtd Paginas --  1630
4 - https://www.capefeargames.com/specials  Qtd Paginas --  2473
5 - https://www.capefeargames.com/products/multi_search  Qtd Paginas --  3383
6 - https://www.capefeargames.com/retail_store  Qtd Paginas --  4223
7 - https://www.capefeargames.com/user/login  Qtd Paginas --  5066
8 - https://www.capefeargames.com/  Qtd Paginas --  5908
9 - https://www.capefeargames.com/advanced_search  Qtd Paginas --  6748
10 - https://www.capefeargames.com/  Qtd Paginas --  7581
11 - https://www.capefeargames.com/catalog/event_registration-magic_events-prerelease_events/4277  Qtd Paginas --  8470
12 - https://www.capefeargames.com/products/search?q=commander+2019  Qtd Paginas --  9382
13 - https://www.capefeargames.com/catalog/magic_singles-throne_of_eldraine/oko_thief_of_crowns/606112  Q

77 - https://www.capefeargames.com/catalog/magic_singles-core_sets-magic_origins/2293  Qtd Paginas --  68511
78 - https://www.capefeargames.com/catalog/magic_singles-core_sets-magic_2015/1453  Qtd Paginas --  69496
79 - https://www.capefeargames.com/catalog/magic_singles-core_sets-magic_2014/871  Qtd Paginas --  70482
80 - https://www.capefeargames.com/catalog/magic_singles-core_sets-magic_2013/774  Qtd Paginas --  71468
81 - https://www.capefeargames.com/catalog/magic_singles-core_sets-magic_2012/702  Qtd Paginas --  72452
82 - https://www.capefeargames.com/catalog/magic_singles-core_sets-magic_2011/298  Qtd Paginas --  73437
83 - https://www.capefeargames.com/catalog/magic_singles-core_sets-magic_2010/224  Qtd Paginas --  74427
84 - https://www.capefeargames.com/catalog/magic_singles-core_sets-10th_edition/76  Qtd Paginas --  75412
85 - https://www.capefeargames.com/catalog/magic_singles-core_sets-9th_edition/75  Qtd Paginas --  76397
86 - https://www.capefeargames.com/catalog/magic_

152 - https://www.capefeargames.com/catalog/magic_singles-odyssey_block-torment/37  Qtd Paginas --  140435
153 - https://www.capefeargames.com/catalog/magic_singles-odyssey_block-judgment/36  Qtd Paginas --  141418
154 - https://www.capefeargames.com/catalog/magic_singles-invasion_block/39  Qtd Paginas --  142399
155 - https://www.capefeargames.com/catalog/magic_singles-invasion_block-invasion/42  Qtd Paginas --  143244
156 - https://www.capefeargames.com/catalog/magic_singles-invasion_block-planeshift/41  Qtd Paginas --  144229
157 - https://www.capefeargames.com/catalog/magic_singles-invasion_block-apocalypse/40  Qtd Paginas --  145211
158 - https://www.capefeargames.com/catalog/magic_singles-masques_block/43  Qtd Paginas --  146194
159 - https://www.capefeargames.com/catalog/magic_singles-masques_block-mercadian_masques/46  Qtd Paginas --  147039
160 - https://www.capefeargames.com/catalog/magic_singles-masques_block-nemesis/45  Qtd Paginas --  148023
161 - https://www.capefeargames

224 - https://www.capefeargames.com/catalog/magic_singles-duel_decks-duel_deck_sorin_vs_tibalt/853  Qtd Paginas --  210530
225 - https://www.capefeargames.com/catalog/magic_singles-duel_decks-duel_deck_speed_vs_cunning/1553  Qtd Paginas --  211551
226 - https://www.capefeargames.com/catalog/magic_singles-duel_decks-duel_deck_venser_vs_koth/761  Qtd Paginas --  212558
227 - https://www.capefeargames.com/catalog/magic_singles-duel_decks-duel_decks_zendikar_vs_eldrazi/2623  Qtd Paginas --  213554
228 - https://www.capefeargames.com/catalog/magic_singles-from_the_vault/294  Qtd Paginas --  214553
229 - https://www.capefeargames.com/catalog/magic_singles-from_the_vault-from_the_vault_angels/2583  Qtd Paginas --  215405
230 - https://www.capefeargames.com/catalog/magic_singles-from_the_vault-from_the_vault_annihilation/1543  Qtd Paginas --  216351
231 - https://www.capefeargames.com/catalog/magic_singles-from_the_vault-from_the_vault_dragons/622  Qtd Paginas --  217297
232 - https://www.cape

290 - https://www.capefeargames.com/catalog/war_of_the_spark_planeswalker_deck_exclusive/4283  Qtd Paginas --  273437
291 - https://www.capefeargames.com/catalog/core_set_2020_planeswalker_deck_exclusives/4396  Qtd Paginas --  274357
292 - https://www.capefeargames.com/catalog/magic_singles-premium_deck_series/268  Qtd Paginas --  275335
293 - https://www.capefeargames.com/catalog/magic_singles-premium_deck_series-premium_deck_series_fire__lightning/554  Qtd Paginas --  276180
294 - https://www.capefeargames.com/catalog/magic_singles-premium_deck_series-premium_deck_series_slivers/282  Qtd Paginas --  277193
295 - https://www.capefeargames.com/catalog/magic_singles-premium_deck_series-premium_deck_series_graveborn/734  Qtd Paginas --  278185
296 - https://www.capefeargames.com/catalog/magic_singles-signature_spellbook_series/4013  Qtd Paginas --  279195
297 - https://www.capefeargames.com/catalog/magic_singles-signature_spellbook_series-signature_spellbooks_jace/3983  Qtd Paginas --  2

364 - https://www.capefeargames.com/catalog/pokemon_singles-xy-flashfire/1943  Qtd Paginas --  344575
365 - https://www.capefeargames.com/catalog/pokemon_singles-xy-xy/1933  Qtd Paginas --  345572
366 - https://www.capefeargames.com/catalog/pokemon_singles-xy-promos_xy_black_star/2153  Qtd Paginas --  346564
367 - https://www.capefeargames.com/catalog/pokemon_singles-xy-evolutions/3243  Qtd Paginas --  347473
368 - https://www.capefeargames.com/catalog/pokemon_singles-xystarpromo/2763  Qtd Paginas --  348464
369 - https://www.capefeargames.com/catalog/pokemon_singles-generations/3193  Qtd Paginas --  349378
370 - https://www.capefeargames.com/catalog/pokemon_singles-black_and_white/2143  Qtd Paginas --  350365
371 - https://www.capefeargames.com/catalog/pokemon_singles-black_and_white-legendary_treasures/2073  Qtd Paginas --  351242
372 - https://www.capefeargames.com/catalog/pokemon_singles-black_and_white-plasma_blast/2033  Qtd Paginas --  352233
373 - https://www.capefeargames.com/c

439 - https://www.capefeargames.com/catalog/yugioh_singles/228  Qtd Paginas --  413256
440 - https://www.capefeargames.com/catalog/yugioh_singles-2019_yugioh/4404  Qtd Paginas --  414125
441 - https://www.capefeargames.com/catalog/yugioh_singles-2019_yugioh-rising_rampage/4394  Qtd Paginas --  414979
442 - https://www.capefeargames.com/catalog/yugioh_singles-2019_yugioh-battles_of_legend_heros_revenge/4390  Qtd Paginas --  415863
443 - https://www.capefeargames.com/catalog/yugioh_singles-2019_yugioh-speed_duel_attack_from_the_deep/4306  Qtd Paginas --  416844
444 - https://www.capefeargames.com/catalog/yugioh_singles-2019_yugioh-dark_neostorm/4290  Qtd Paginas --  417722
445 - https://www.capefeargames.com/catalog/yugioh_singles-2019_yugioh-structure_deck_order_of_the_spellcasters/4359  Qtd Paginas --  418610
446 - https://www.capefeargames.com/catalog/yugioh_singles-2019_yugioh-duel_power/4383  Qtd Paginas --  419488
447 - https://www.capefeargames.com/catalog/yugioh_singles-2019_yugi

### Tabela com informações do Crawler Baseline

In [ ]:
data = pd.DataFrame(Dados,columns=['Número','Link','Site'])
data

## * Heurística